In [1]:
import sys
import numpy as np
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [3]:
# nnet_path = '../artifacts/models/model_v3.2.0/model.h5'
# data_csv = '../data/v3.2.0/test.csv'
# all_samples = TOTUtils.load_samples(data_csv, frac=0.1)
# samples = TOTUtils.filter_samples(all_samples, nnet_path)
# print(f'{len(samples)} correct samples')

13528 correct samples


In [397]:
from robustness import find_local_robustness_boundaries

h5_path = '../artifacts/models/model_v3.2.0/model.h5'
sample = ([0.23926416842332435, -0.15174888439921375, -0.12715716851930392, -0.06499066379491034, -0.044038348638094, -0.044549369363895516, -2.1657892246029085, -0.14904223875562067, 1.7803860411643386, -1.8489723834384288, -1.4227079179070772, -1.146822736559327, -0.019598968186462968, -0.497276625704834, -0.059858693698986526, -1.1658575654628252, 0.5013980886462887, -0.12424305809637555, -0.16772960456016833, 0.8476144787315946, -0.04599664704894778, 0.017279279428270938, 0.18448958004575428, -0.014432784637498619, -0.16232216451525688], [0, 0, 0, 0, 1])
results = find_local_robustness_boundaries(h5_path, [sample], d_min=0.00001, d_max=100)
summary, _ = results
print(f'summary: {summary}')

summary: (-0.04372000000000003, 0.21464)


In [394]:
import pandas as pd
from robustness import verify_local_robustness_boundaries

n_inputs = 25
p = 0
nnet_path = '../network/models/v3.2.0/model.nnet'
samples_path = '../logs/robustness/samples.csv'
dists_path = '../logs/robustness/local_summary.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:n_inputs].tolist(), s[n_inputs:].tolist()) for s in samples.values]
distances = pd.read_csv(dists_path, usecols=['dneg', 'dpos']).values

# distances = tuple(distances[0])
# print(f'verifying local robustness dists: {distances}')
# counterexamples = verify_local_robustness_boundaries(nnet_path, samples, distances, ignore_samples=[], timeout=10, verbose=0)
# print(f'counterexamples in {len(counterexamples.keys())} fields')

# for i,ce in counterexamples.items():
#     vals, samp = ce
#     ce_inputs, ce_outputs = vals
#     s_inputs, s_outputs = samp
#     for x in range(25):
#         print(f'x{x}: ', abs(ce_inputs[x])-abs(s_inputs[x]))

details = pd.read_csv('../logs/robustness/local_details.csv', usecols=['dneg','dpos'])
dneg = details[details['dneg'] < 0]['dneg']
dpos = details[details['dpos'] > 0]['dpos']
print(dneg.max(), dpos.min())

-0.0013379999999999993 0.000401


In [230]:
# incorrect_samples = [samples[51], samples[209], samples[887], samples[1004], samples[1127], samples[1195], samples[1222], samples[1289]]
# p=0.000141
# dists = (distances[0]+p, distances[1]-p)
# s=1289
# p=0.0001
# sample = samples[s]

# examps = verify_local_robustness_boundaries(nnet_path, [sample], dists, precision_adjustment=0, verbose=1)
# print(examps)

# for i,examp in examps.items():
#     vals, samp = examp
#     ce_inputs, ce_outputs = vals
#     s_inputs, s_outputs = samp
#     for x in range(25):
#         print(f'x{x}: ', abs(ce_inputs[x])-abs(s_inputs[x]))

INFO:robustness:all distances valid (34ms)
{}


In [400]:
from tot_net import TOTNet

nnet_path = '../artifacts/models/model_v3.2.0/model.nnet'
sample = ([0.23926416842332435, -0.15174888439921375, -0.12715716851930392, -0.06499066379491034, -0.044038348638094, 0.3313194507444112, 0.4782613721223489, 0.2327204327318842, 0.14517771117862072, 1.3552479661059729, 2.24086091849637, 0.87197434103912, 0.4850011890061274, 0.577800190511209, 1.0597868699177289, -0.39768393057496987, -0.362047643207948, -0.12424305809637555, -0.16772960456016833, -1.778873779785491, -0.04599664704894778, -0.5208565928982523, 0.18448958004575428, 0.5282850327992676, -0.16232216451525688], [0, 0, 1, 0, 0])
n_inputs=25
n_outputs=5
negd, posd = (-0.0024129999999999998, 0.007748000000000001)

inputs, outputs = sample
expected_category = outputs.index(max(outputs))
other_categories = [y for y in range(n_outputs) if y != expected_category]
lbounds = [x+negd for x in inputs]
ubounds = [x+posd for x in inputs]
net = TOTNet(nnet_path)
for oc in other_categories:
    net.reset_query()
    net.set_lower_bounds(lbounds)
    net.set_upper_bounds(ubounds)
    net.set_expected_category(oc)
    vals, _ = net.solve(timeout=10)
    counter_inputs, counter_outputs = vals
    if len(counter_inputs) > 0 or len(counter_outputs) > 0:
        print('Counterexample:\n', vals)
        break

Counterexample:
 ([0.1955441684233243, -0.18514550681258415, 0.08748283148069608, -0.10871066379491037, 0.170601651361906, 0.057229047730204624, -1.9511492246029085, -0.1927622387556207, 1.7366660411643386, -1.634332383438429, -1.4664279179070772, -0.9321827365593272, -0.063318968186463, -0.540996625704834, -0.10357869369898656, -1.2095775654628251, 0.4576780886462886, -0.026350308813991488, 0.002729866340970037, 0.8038944787315946, -0.0897166470489478, 0.23191927942827092, 0.2921540096345387, 0.20020721536250138, 0.052317835484743114], [-0.790979, 0.121041, -0.254524, 0.100963, 0.0369568])
